In [1]:
import numpy as np
from astropy import units as u 
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import QTable
from reproject import reproject_interp
from astropy import stats
import os 
from astropy.table import QTable, join, vstack, hstack, Column


plt.style.use('paper.mplstyle')

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [3]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                if '628e' in file: 
                    continue
                else: 
                    print(os.path.join(root, file))
                    file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha_sic.fits', root_dir)
files_nebs_ = get_files('_nebmask.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/ngc1087_hst_ha_sic.fits
/Users/abarnes/Dropbox/work/Smallprojects/ga

In [5]:
files_nebs = []
for file_maps in files_maps: 
    for file_nebs in files_nebs_: 
        if file_maps.split(root_dir)[-1].split('/')[0] == file_nebs.split(root_dir)[-1].split('/')[0]:
            print(file_maps, file_nebs)
            files_nebs += [file_nebs]
            break

if len(files_maps) != len(files_nebs):
    print('WARNING: DIFFERENT NUMBER OF FILES')

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha_sic.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/muse/NGC3627_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha_sic.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/muse/NGC4254_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha_sic.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/muse/NGC1512_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha_sic.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/muse/NGC1433_nebmask.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha_sic.fits /Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/muse/NGC5068_neb

In [6]:
output_stats = []

for file_maps, file_nebs in zip(files_maps, files_nebs): 

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    muscat_hdu = fits.open(file_nebs)[0]

    # Update arrays
    muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
    muscat_hdu.data[muscat_hdu.data==-1] = np.nan

    # Interpolate masks
    muscat_data_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header)
    muscat_data_mask = ~np.isnan(muscat_data_re)
    data_outmask = hstha_hdu.data[~muscat_data_mask]

    # Get RMS for whole map... 
    std = stats.mad_std(data_outmask, ignore_nan=True)  # Get noise
    std = stats.mad_std(data_outmask[data_outmask<20*std], ignore_nan=True)  
    mean = np.nanmean(data_outmask[data_outmask<20*std])  
    median = np.nanmedian(data_outmask[data_outmask<20*std])  
    print(std, mean, median)
    output_stats += [[std, mean, median]]

104.19066496276069 20.455137 17.919247
116.52447842590125 24.253899 21.304026
97.14608739883653 8.452168 6.8892026
89.60047326517433 7.658343 5.0533876
107.32039309108784 14.663575 12.013839
92.52699286036356 7.554715 5.776401
86.74731347748111 14.996686 11.78062
112.6206010068444 17.385052 15.513111
94.22991203985963 8.879118 6.7230635
81.67794208496485 11.648807 9.424653
94.02981412996763 8.5234785 6.3709226
118.91419493970301 8.067088 5.8374014
91.6787315568802 12.29103 10.132849
98.86243881507635 -1.3010136 -2.7290134
38.43740588195304 13.72786 11.398844
33.684780771050725 10.5704155 7.9639378
23.091591482905738 4.5896654 3.8047748
95.62139561997434 5.5193486 3.6181595


In [7]:
# Get into a astropy table the output of the stats of the maps to .tex
gals = []
for file_maps in files_maps:
    gal = file_maps.split(root_dir)[-1].split('/')[0]
    gals += [gal]

output_stats = np.array([output_stats][0], dtype=np.float64)
gals = np.array(gals)

for i, gal in enumerate(gals):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628'
    gals[i] = gal

table1 = Column(gals, name='gal_name')
table2 = QTable(output_stats, names=('std', 'mean', 'median'))

table = hstack([table1, table2])
table.sort('gal_name')
table

gal_name,std,mean,median
str8,float64,float64,float64
IC~5332,91.6787315568802,12.291029930114746,10.132848739624023
NGC~0628,38.43740588195304,13.727860450744629,11.398843765258789
NGC~1087,94.22991203985963,8.879117965698242,6.7230634689331055
NGC~1300,23.091591482905738,4.589665412902832,3.804774761199951
NGC~1365,86.74731347748111,14.996685981750488,11.780619621276855
NGC~1385,94.02981412996763,8.523478507995605,6.370922565460205
NGC~1433,89.60047326517433,7.6583428382873535,5.053387641906738
NGC~1512,97.14608739883653,8.452168464660645,6.88920259475708
NGC~1566,95.62139561997434,5.519348621368408,3.618159532546997


In [8]:
formats={
        'gal_name': '%s',
        'std': '%0.1f',
        'mean': '%0.1f',
        'median': '%0.1f'}

table.write('./tabs/galmap_props.tex', formats=formats, overwrite=True)

In [9]:
# Reorganise to match names in .fits files
gals = table['gal_name']

for i, gal in enumerate(gals):
    if '~' in gal:
        gal_split = gal.split('~')
        gal = gal_split[0] + gal_split[1]
    table['gal_name'][i] = gal

table.write('./tabs/galmap_props.fits', overwrite=True)
table

gal_name,std,mean,median
str8,float64,float64,float64
IC5332,91.6787315568802,12.291029930114746,10.132848739624023
NGC0628,38.43740588195304,13.727860450744629,11.398843765258789
NGC1087,94.22991203985963,8.879117965698242,6.7230634689331055
NGC1300,23.091591482905738,4.589665412902832,3.804774761199951
NGC1365,86.74731347748111,14.996685981750488,11.780619621276855
NGC1385,94.02981412996763,8.523478507995605,6.370922565460205
NGC1433,89.60047326517433,7.6583428382873535,5.053387641906738
NGC1512,97.14608739883653,8.452168464660645,6.88920259475708
NGC1566,95.62139561997434,5.519348621368408,3.618159532546997
